In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy

In [2]:
def get_aljazeera():
    driver = webdriver.Chrome()
    url= 'https://www.aljazeera.com/'
    driver.get(url)
    cookies_accept = driver.find_element(By.CLASS_NAME, "icon--search").click()
    time.sleep(5)
    search_container= driver.find_element(By.CLASS_NAME, 'search-bar__input-container').click
    time.sleep(3)
    search_bar     = driver.find_element(By.CLASS_NAME, 'search-bar__input')
    search_bar.send_keys('Ukraine war')
    search_bar.send_keys(Keys.RETURN)
    time.sleep(3)
    lnks=driver.find_elements(By.TAG_NAME,"a")
    lst_aljazeera=[lnk.get_attribute('href') for lnk in lnks]
    lst_aj=[url for url in lst_aljazeera if '/news/2' in url ]
    return lst_aj

In [3]:
def create_dict_aj(lst_aj):
    dict_aj={}
    for url in lst_aj:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        article= soup.getText()
        article = article[article.find('facebooktwitterwhatsapp'):]
        article = article[:article.find('Source: Al Jazeera')]
        article = article.replace('\n','').replace('\xa0',' ').replace('facebooktwitterwhatsapp','')
        dict_aj[url]= article
    return dict_aj

In [8]:
def descriptives(dict_aj):
    length=[len(articles) for articles in dict_aj.values()]
    ukraine=[articles.lower().count('ukraine') for articles in dict_aj.values()]
    war=[articles.lower().count('war') for articles in dict_aj.values()]
    russia=[articles.lower().count('russia') for articles in dict_aj.values()]
    putin = [articles.lower().count('putin') for articles in dict_aj.values()]
    zelensky= [articles.lower().count('zelensky') for articles in dict_aj.values()]
    dict_length= {'length':length,'ukraine':ukraine,'war':war, 'russia':russia,'putin':putin,'zelensky':zelensky}
    df_aj_descr= pd.DataFrame(dict_length)
    return df_aj_descr

In [4]:
def create_df_aj(dict_aj):
    idx= {'article'}
    df_aj= pd.DataFrame(dict_aj,idx).transpose().reset_index()
    df_aj= df_aj.rename(columns={"index": "link"})
    return df_aj

In [5]:
lst_aj = get_aljazeera()

In [6]:
dict_aj = create_dict_aj(lst_aj)

In [10]:
df_aj_descr= descriptives(dict_aj)

In [11]:
df_aj= create_df_aj(dict_aj)

In [12]:
df_aj_all =df_aj.join(df_aj_descr)

In [13]:
df_aj_all

,link,article,length,ukraine,war,russia,putin,zelensky
0,https://www.aljazeera.com/news/2023/2/26/russi...,"Here is the situation as it stands on Sunday, ...",1861,5,4,11,0,0
1,https://www.aljazeera.com/news/2023/2/16/russi...,Ukraine faced its toughest week so far this ye...,9690,14,8,35,0,2
2,https://www.aljazeera.com/news/2023/2/21/qa-dr...,"Moscow, Russia – As the first anniversary of t...",11422,21,14,55,5,0
3,https://www.aljazeera.com/news/2023/2/24/russi...,"Here is the situation as it stands on Friday, ...",2218,9,4,9,1,1
4,https://www.aljazeera.com/news/2022/2/28/russi...,"As the Russian offensive enters its 368th day,...",7633,27,4,48,3,0
5,https://www.aljazeera.com/news/2023/1/22/russi...,"Here is the situation as it stands on Sunday, ...",2332,5,1,6,0,2
6,https://www.aljazeera.com/news/2022/8/31/is-uk...,Ukraine’s armed forces claim to have launched ...,6873,12,4,27,0,1
